# Characteristics of Businesses - Group Exercise

In [1]:
import pandas as pd
import requests

tech_variables = 'https://api.census.gov/data/2018/abstcb/variables.html'
owner_variables = 'https://api.census.gov/data/2018/abscbo/variables.html'
characteristics_variables = 'https://api.census.gov/data/2018/abscb/variables.html'
company_summary_variables = 'https://api.census.gov/data/2018/abscs/variables.html'


### Get Tables of Variables

In [2]:
tech_vars = pd.read_html(tech_variables)[0]
owner_vars = pd.read_html(owner_variables)[0]
characteristic_vars = pd.read_html(characteristics_variables)[0]
company_summary_vars = pd.read_html(company_summary_variables)[0]

tech_vars = tech_vars[[_ for _ in tech_vars.columns[:2]]]
owner_vars = owner_vars[[_ for _ in owner_vars.columns[:2]]]
characteristic_vars = characteristic_vars[[_ for _ in characteristic_vars.columns[:2]]]
company_summary_vars = company_summary_vars[[_ for _ in company_summary_vars.columns[:2]]] 

tech_vars.rename(columns = {'Label': 'Tech Labels'}, inplace = True)
owner_vars.rename(columns = {'Label': 'Owner Labels'}, inplace = True)
characteristic_vars.rename(columns = {'Label': 'Characteristic Labels'}, inplace = True)
company_summary_vars.rename(columns = {'Label': 'Company Summary Labels'}, inplace = True)

In [3]:
grouped_tables = pd.merge(tech_vars, owner_vars, left_on = 'Name', right_on = 'Name', how = 'outer')
grouped_tables = pd.merge(grouped_tables, characteristic_vars, left_on = 'Name', right_on = 'Name', how = 'outer')
grouped_tables = pd.merge(grouped_tables, company_summary_vars , left_on = 'Name', right_on = 'Name', how = 'outer')
grouped_tables.fillna("-", inplace = True)
grouped_tables = grouped_tables[(~grouped_tables['Tech Labels'].str.contains('Standard error|standard error'))]
grouped_tables = grouped_tables[(~grouped_tables['Owner Labels'].str.contains('Standard error|standard error'))]
grouped_tables = grouped_tables[(~grouped_tables['Characteristic Labels'].str.contains('Standard error|standard error'))]
grouped_tables = grouped_tables[(~grouped_tables['Company Summary Labels'].str.contains('Standard error|standard error'))]

grouped_tables.reset_index(drop = True, inplace = True)
grouped_tables = grouped_tables[:-2]

In [4]:
# grouped_tables.sort_values(by = 'Name', ascending = False)
vars_of_interest = ['NAICS2017','YIBSZFI','SEX','QDESC','NSFSZFI','GEO_ID','RACE_GROUP','BUSCHAR']
target_subset = grouped_tables[(grouped_tables['Name'].isin(vars_of_interest))]
target_subset = target_subset[['Name','Company Summary Labels','Characteristic Labels','Owner Labels','Tech Labels']]
print(target_subset)

          Name      Company Summary Labels         Characteristic Labels  \
8       GEO_ID  Geographic identifier code    Geographic identifier code   
17   NAICS2017             2017 NAICS code               2017 NAICS code   
19     NSFSZFI                           -                             -   
22  RACE_GROUP                   Race code                     Race code   
25         SEX                    Sex code                      Sex code   
42       QDESC                           -     Question description code   
44     BUSCHAR                           -  Business characteristic code   
47     YIBSZFI      Years in business code        Years in business code   

                  Owner Labels                    Tech Labels  
8   Geographic identifier code     Geographic identifier code  
17             2017 NAICS code                2017 NAICS code  
19                           -  Employment size of firms code  
22                           -                      Race co

In [5]:
variable_dict = {}
for i,label in enumerate(target_subset.columns[1:]):
    variable_list = []
    for item in target_subset[(target_subset[label] != "-")].Name.tolist():
        variable_list.append(item)
        variable_list.append(f'{item}_LABEL')
    variable_list = [_ for _ in variable_list if _ not in ['GEO_ID_LABEL', 'QDESC_LABEL']]
    in_table = "NAME," + ",".join(variable_list)
    variable_dict[i] = in_table
variable_dict


{0: 'NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL,YIBSZFI,YIBSZFI_LABEL',
 1: 'NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL,QDESC,BUSCHAR,BUSCHAR_LABEL,YIBSZFI,YIBSZFI_LABEL',
 2: 'NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,QDESC',
 3: 'NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,NSFSZFI,NSFSZFI_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL'}

In [6]:
# Might be able to pass in the same vars for every call, and if it exists it will be returned. Not sure if this will work.
# otherwise, store the variables in a dictionary where the key is 0-3 and the values are a string of the variables

industry_code = '61'
qdesc1 = 'B27'
# links = [
#     f'https://api.census.gov/data/2018/abscs?get={variable_dict[0]}&for=state:*&NAICS2017={industry_code}',
#     f'https://api.census.gov/data/2018/abscb?get={variable_dict[1]}&for=state:*&NAICS2017={industry_code}&BUSCHAR&QDESC={qdesc1}',
#     f'https://api.census.gov/data/2018/abscbo?get={variable_dict[2]}&for=state:*&OWNCHAR=CG&NAICS2017={industry_code}&QDESC={qdesc1}',
#     f'https://api.census.gov/data/2018/abstcb?get={variable_dict[3]}&for=state:*&NAICS2017={industry_code}'
# ]

links = [
    f'https://api.census.gov/data/2018/abscs?get={variable_dict[0]}&for=state:*',
    f'https://api.census.gov/data/2018/abscb?get={variable_dict[1]}&for=state:*',
    f'https://api.census.gov/data/2018/abscbo?get={variable_dict[2]}&for=state:*',
    f'https://api.census.gov/data/2018/abstcb?get={variable_dict[3]}&for=state:*'
]

def get_data_frame(url):
    return pd.read_csv(url)
    
for link in links:
    print(link)
# Commented out so we don't use up calls before we have everything right.
# comp_sum_df = get_data_frame(links[0])
# bus_char_df = get_data_frame(links[1])
# bus_own_df = get_data_frame(links[2])
# bus_tech_df = get_data_frame(links[3])
  
    
    
    
    

https://api.census.gov/data/2018/abscs?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL,YIBSZFI,YIBSZFI_LABEL&for=state:*
https://api.census.gov/data/2018/abscb?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL,QDESC,BUSCHAR,BUSCHAR_LABEL,YIBSZFI,YIBSZFI_LABEL&for=state:*
https://api.census.gov/data/2018/abscbo?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,QDESC&for=state:*
https://api.census.gov/data/2018/abstcb?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,NSFSZFI,NSFSZFI_LABEL,RACE_GROUP,RACE_GROUP_LABEL,SEX,SEX_LABEL&for=state:*
